# Plotting with Matplotlib Animations

The general steps are as follows: 

1. Requires packages: ffmpeg, IPython, matplotlib
2. Import libraries
3. Create the fig, ax to store the plot
4. Set the plot limits for the x and y axes
5. Initialize the plot elements (line object in the example that follows; unpacks a tuple returned by plot)
6. Set up the plot details (labels, etc.)
7. Define the initialization function (store the background of the animation in each frame)
8. Define the animation function (takes each frame and updates the plot elements)
9. Create the animation object (call the FuncAnimation method)
10. Save (and embed - for Jupyter notebooks).

## Plot Example: Sine Wave

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Video

with plt.style.context('dark_background'):
    # Create a figure and axis where the frames will be displayed
    fig, ax = plt.subplots()
    
    # Set up plot limits
    ax.set_xlim(0, 2 * np.pi)
    ax.set_ylim(-2, 2)

    # Initialize a line object
    line, = ax.plot([], [], lw=2)

    # Set up plot details that don't need to update with each frame
    ax.set_xlabel("Angle [rad]")
    ax.set_ylabel("Amplitude")

    # Function to initialize the background of the animation
    def init():
        line.set_data([], [])
        return line,

    # Animation function called sequentially
    def animate(frame):
        x = np.linspace(0, 2 * np.pi, 1000)
        y = np.sin(x + 0.1 * frame)
        line.set_data(x, y)
        ax.set_title(f"Sine Wave Animation - Frame {frame}") # Add plot details that updte with each frame
        return line,

# Create animation object
ani = animation.FuncAnimation(
    fig, animate, init_func=init, frames=100, interval=20, blit=True
)

# Save the animation as an MP4 using ffmpeg
ani.save("Anim.mp4", writer="ffmpeg", fps=30)

# Display the saved video in Jupyter Notebook (optional)
video=Video("Anim.mp4", embed=True)
display(video)

# close fig to prevent Jupyter from displaying a static image
plt.close(fig)

## Plot Example: Simple 1D Random Walk 

This plot animates the position and RMSD of a simple random walk over its duration. Includes animation of multiple plots in the same panel. 

In [34]:
# Position and Variance of a simple 1D random walk

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import Video

# Parameters
num_steps=10**3 # The number of steps in the walk
time_step = 0.1 # Time required for one full step
t=int(num_steps * time_step)

# Model

steps=np.random.choice([-1, 1], size=t)

position=np.cumsum(steps)

squared_displacement=position ** 2
variance=np.cumsum(squared_displacement / np.arange(1, t + 1))


# Animated Plots
with plt.style.context("dark_background"):
    # Set up the figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Initialize lines for position and variance
    line1, = ax1.plot([], [], lw=2)
    line2, = ax2.plot([], [], lw=2)

    # Set up the axes limits and labels
    ax1.set_xlim(0, t)
    ax1.set_ylim(position.min() * 1.1, position.max() * 1.1)
    ax1.set_title("1D Random Walk")
    ax1.set_xlabel("Time (s)")
    ax1.set_ylabel("Position")

    ax2.set_xlim(0, t)
    ax2.set_ylim(0, variance.max() * 1.1)
    ax2.set_title("Variance of Position")
    ax2.set_xlabel("Time (s)")
    ax2.set_ylabel("Variance")

    # Initialization function to clear the data at the start of the animation
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        return line1, line2

    # Update function for each frame
    def update(frame):
        # Update position line (up to current frame)
        line1.set_data(np.arange(frame), position[:frame])
    
        # Update variance line (up to current frame)
        line2.set_data(np.arange(frame), variance[:frame])
    
        return line1, line2

    # Create the animation
    ani = FuncAnimation(fig, update, frames=np.arange(1, t, int(t / 100)),  # Use a subset of frames
                    init_func=init, blit=False, interval=10)


    # Save the animation as an MP4 using ffmpeg
    ani.save("random_walk_dark.mp4", writer="ffmpeg", fps=30)

# Display the saved video in Jupyter Notebook (optional)
video = Video("random_walk_dark.mp4", embed=True); # Include the ; to suppress the fig from being automatically displayed by Jupyter
display(video)


plt.close(fig)

## Plot Example: Conway's Game of Life

This example produces a 2D image changes at each step following a convolution procedure.

In [47]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from matplotlib.animation import FuncAnimation
from IPython.display import Video

def initialize_grid(size):
    # Initialize the grid
    return np.random.choice([0, 1], size=size*size, p=[0.9, 0.1]).reshape(size, size)

def update_grid(grid):
    # Update the grid based on the GOL rules
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    neighbors = convolve2d(grid, kernel, mode='same', boundary='wrap')
    return (neighbors == 3) | ((grid == 1) & (neighbors == 2))

def update_plot(frame_num, img, grid):
    # Update the plot for animation
    new_grid = update_grid(grid)
    img.set_data(new_grid)
    grid[:] = new_grid 
    return img,

# Set the size of the grid and the number of frames for the animation
size = 100
num_frames = 1000  

# Initialize the grid
grid = initialize_grid(size)

# Setup the plot
fig, ax = plt.subplots()
img = ax.imshow(grid, cmap='binary')
ax.axis('off')

# Create and save the animation
ani = FuncAnimation(fig, update_plot, frames=num_frames, fargs=(img, grid), blit=True)
ani.save('game_of_life.mp4', writer='ffmpeg', fps=10)

# Display the saved video in Jupyter Notebook (optional)
video = Video("game_of_life.mp4", embed=True); # Include the ; to suppress the fig from being automatically displayed by Jupyter
display(video)

plt.close(fig)

## Damped Harmonic Oscillator Phase Space

The behavior of a simple harmonic oscillator (such as a pendulum with a small swing) can be described using a piecewise first-order differential equation: 

$$
\begin{cases} \frac{dx}{dt} = v \\ \frac{dv}{dt}=-\omega ^2 x \end{cases}
$$

where, $x$ is position, $v$ is velocity, and $\omega$ is angular velocity. 

A damped oscillator loses energy over time, typically due to friction. It can be described via the following system of first-order differential equations: 

$$
\begin{cases} \frac{dx}{dt} = v \\ \frac{dv}{dt}=-2 \beta v -\omega ^2 x \end{cases}
$$

where, $x$ is position, $v$ is velocity, $\omega$ is angular velocity, and $\beta$ is the damping coefficient.


$$
\begin{cases} \beta < \omega_o  & \text{Exponentially decreasing amplitude} \\ \beta = \omega_o & \text{Returns to equilibrium without oscillating} \\
\beta > \omega_o & \text{Returns to equilibrium without oscillating, but quicker than the critically damped case} \end{cases}
$$

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.integrate import odeint
from IPython.display import Video

# Natural angular frequency
omega_0 = 1.0

# Damping coefficient
beta = .05  # Adjust this value for different damping scenarios; try 0, 1.0, and 2.0

# System of differential equations
def damped_harmonic_oscillator(state, t):
    x, v = state
    dxdt = v
    dvdt = -2 * beta * v - omega_0**2 * x
    return [dxdt, dvdt]

# Create a grid of x and v values
x_vals = np.linspace(-5, 5, 20)
v_vals = np.linspace(-5, 5, 20)
X, V = np.meshgrid(x_vals, v_vals)

# Compute derivatives at each point in the grid
DX = V
DV = -2 * beta * V - omega_0**2 * X

plt.style.use('dark_background')

# Initialize the plot
fig, ax = plt.subplots(figsize=(5, 3))

# Plot the vector field using quiver
ax.quiver(X, V, DX, DV, color='gray', alpha=0.5)

# Set plot limits
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)

# Set labels
ax.set_xlabel('Position x')
ax.set_ylabel('Velocity v')
ax.set_title('Phase Space Trajectory of a Damped Harmonic Oscillator')

# Initialize a line object for the trajectory
trajectory_line, = ax.plot([], [], 'r-', lw=2)

# Initialization function
def init():
    trajectory_line.set_data([], [])
    return trajectory_line,

# Time points for integration
t = np.linspace(0, 50, 1000)

# Initial condition
initial_state = [4.0, 0.0]

# Solve the differential equation
solution = odeint(damped_harmonic_oscillator, initial_state, t)
x_solution = solution[:, 0]
v_solution = solution[:, 1]

# Animation function
def animate(i):
    # Update the trajectory line
    trajectory_line.set_data(x_solution[:i], v_solution[:i])
    return trajectory_line,

# Create the animation
ani = animation.FuncAnimation(
    fig, animate, init_func=init, frames=len(t), interval=20, blit=False
)
# Save the animation (optional)
ani.save('damped_harm_osc.mp4', writer='ffmpeg', fps=30)

video = Video('damped_harm_osc.mp4', embed=True)
display(video)

# Close the figure to prevent static plot display (optional)
plt.close(fig)
